In [ ]:
!ls ../input/uploads/tryp_dataset/

In [44]:
import os
import cv2
import numpy as np

dataset_path = '../input/uploads/tryp_dataset/'
train_norm = 'train/norm'
train_trypo = 'train/trypo'
valid_norm = 'valid/norm'
valid_trypo = 'valid/trypo'

train_data_ratio = 1 # ratio of training data

train_norm_path = os.path.join(dataset_path, train_norm)
train_norm_filelist = os.listdir(train_norm_path)
train_norm_filelist = train_norm_filelist[:int(train_data_ratio * len(train_norm_filelist))]

train_trypo_path = os.path.join(dataset_path, train_trypo)
train_trypo_filelist = os.listdir(train_trypo_path)
train_trypo_filelist = train_trypo_filelist[:int(train_data_ratio * len(train_trypo_filelist))]

valid_norm_path = os.path.join(dataset_path, valid_norm)
valid_norm_filelist = os.listdir(valid_norm_path)

valid_trypo_path = os.path.join(dataset_path, valid_trypo)
valid_trypo_filelist = os.listdir(valid_trypo_path)

def get_images(file_list, path):
    return [cv2.imread(os.path.join(path, image)) for image in file_list]

train_norm_images = get_images(train_norm_filelist, train_norm_path)
train_trypo_images = get_images(train_trypo_filelist, train_trypo_path)
valid_norm_images = get_images(valid_norm_filelist, valid_norm_path)
valid_trypo_images = get_images(valid_trypo_filelist, valid_trypo_path)

#print(train_norm_images[0])
#print(train_trypo_images[0])
#print(valid_norm_images[0])
#print(valid_trypo_images[0])


def resize_images(images, size=(224, 224)):
    for i in range(len(images)):
        images[i] = cv2.resize(images[i], size).astype(np.float32)
    return images

train_norm_images = resize_images(train_norm_images)
train_trypo_images = resize_images(train_trypo_images)
valid_norm_images = resize_images(valid_norm_images)
valid_trypo_images = resize_images(valid_trypo_images)

In [45]:
from random import shuffle
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils

def process_images(images):
    return np.divide(images, 255)
   
def get_data(norm, trypo):
    norm = process_images(np.array(norm))
    trypo = process_images(np.array(trypo))
    X = np.concatenate([norm, trypo])
    Y1 = np.zeros(len(norm))
    Y2 = np.ones(len(trypo))
    Y = np.concatenate([Y1, Y2])
    Y = np_utils.to_categorical(Y)
    return X, Y
   
X_train, y_train = get_data(train_norm_images, train_trypo_images)
X_val, Y_val = get_data(valid_norm_images, valid_trypo_images)
print(y_train.shape)

(15884, 2)


In [46]:
X_train.shape
X_train[0].shape
x_size, y_size = 224, 224

X_train.shape

(15884, 224, 224, 3)

In [47]:
x_size, y_size = 224, 224
X_train.shape

(15884, 224, 224, 3)

In [19]:
import keras
model = keras.applications.resnet50.ResNet50(include_top=True, weights=None, classes=2)

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 230, 230, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [14]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
15884/15884 [==============================] - 553s - loss: 0.5721 - acc: 0.7203   
Epoch 2/10
15884/15884 [==============================] - 549s - loss: 0.4988 - acc: 0.7519   
Epoch 3/10
15884/15884 [==============================] - 549s - loss: 0.4695 - acc: 0.7737   
Epoch 4/10
15884/15884 [==============================] - 549s - loss: 0.4638 - acc: 0.7905   
Epoch 5/10
15884/15884 [==============================] - 549s - loss: 0.4252 - acc: 0.8060   
Epoch 6/10
15884/15884 [==============================] - 549s - loss: 0.4673 - acc: 0.7793   
Epoch 7/10
15884/15884 [==============================] - 549s - loss: 0.4182 - acc: 0.8075   
Epoch 8/10
15884/15884 [==============================] - 549s - loss: 0.4025 - acc: 0.8184   
Epoch 9/10
15884/15884 [==============================] - 548s - loss: 0.3968 - acc: 0.8223   
Epoch 10/10
15884/15884 [==============================] - 548s - loss: 0.3850 - acc: 0.8318   


Let's try some mnist

In [35]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D

def larger_convolutional_model():
    # create model
    model = Sequential()
    # Convolutional layer with 30 feature maps of size 5x5
    model.add(Conv2D(30, (5, 5), input_shape=(x_size, y_size, 3), activation='relu'))
    # Pooling layer taking the max over 2*2 patches
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convolutional layer with 15 feature maps of size 3x3
    model.add(Conv2D(15, (3, 3), activation='relu'))
    # Pooling layer taking the max over 2*2 patches
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Dropout layer with a probability of 20%
    model.add(Dropout(0.2))
    # Flatten layer
    model.add(GlobalAveragePooling2D())
    # Fully connected layer with 128 neurons and rectifier activation
    model.add(Dense(128, activation='relu'))
    # Fully connected layer with 50 neurons and rectifier activation
    model.add(Dense(50, activation='relu'))
    # Output layer
    model.add(Dense(2, activation='softmax'))
    
    #compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model    

In [36]:
model = larger_convolutional_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 220, 220, 30)      2280      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 110, 110, 30)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 108, 108, 15)      4065      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 54, 54, 15)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 54, 54, 15)        0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 15)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               2048      
__________

In [37]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, Y_val))

Train on 1587 samples, validate on 1000 samples
Epoch 1/10
1587/1587 [==============================] - 6s - loss: 0.6455 - acc: 0.6307 - val_loss: 0.6963 - val_acc: 0.5000
Epoch 2/10
1587/1587 [==============================] - 6s - loss: 0.6175 - acc: 0.6389 - val_loss: 0.6580 - val_acc: 0.5360
Epoch 3/10
1587/1587 [==============================] - 6s - loss: 0.5974 - acc: 0.6503 - val_loss: 0.6192 - val_acc: 0.6280
Epoch 4/10
1587/1587 [==============================] - 6s - loss: 0.5809 - acc: 0.6868 - val_loss: 0.5973 - val_acc: 0.6470
Epoch 5/10
1587/1587 [==============================] - 6s - loss: 0.5683 - acc: 0.6887 - val_loss: 0.5756 - val_acc: 0.7010
Epoch 6/10
1587/1587 [==============================] - 6s - loss: 0.5601 - acc: 0.7051 - val_loss: 0.5613 - val_acc: 0.7270
Epoch 7/10
1587/1587 [==============================] - 6s - loss: 0.5517 - acc: 0.7045 - val_loss: 0.5491 - val_acc: 0.7420
Epoch 8/10
1587/1587 [==============================] - 6s - loss: 0.5549 - a

In [ ]:
model.evaluate(X_val, Y_val)

Another solution

In [58]:
from keras.layers import MaxPool2D

model = Sequential()

model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(x_size, y_size, 3)))
model.add(MaxPool2D())

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D())

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D())

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D())

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D())

model.add(Conv2D(64, (1, 1), activation='relu'))
model.add(GlobalAveragePooling2D())

model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 24, 24, 64)        36928     
__________

In [59]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, Y_val))

Train on 15884 samples, validate on 1000 samples
Epoch 1/20
15884/15884 [==============================] - 52s - loss: 0.5255 - acc: 0.7324 - val_loss: 0.5183 - val_acc: 0.7420
Epoch 2/20
15884/15884 [==============================] - 52s - loss: 0.4428 - acc: 0.7873 - val_loss: 0.4172 - val_acc: 0.8000
Epoch 3/20
15884/15884 [==============================] - 51s - loss: 0.4014 - acc: 0.8167 - val_loss: 0.5469 - val_acc: 0.7070
Epoch 4/20
15884/15884 [==============================] - 52s - loss: 0.3768 - acc: 0.8308 - val_loss: 0.3778 - val_acc: 0.8290
Epoch 5/20
15884/15884 [==============================] - 52s - loss: 0.3530 - acc: 0.8484 - val_loss: 0.3796 - val_acc: 0.8110
Epoch 6/20
15884/15884 [==============================] - 52s - loss: 0.3442 - acc: 0.8491 - val_loss: 0.3888 - val_acc: 0.8130
Epoch 7/20
15884/15884 [==============================] - 52s - loss: 0.3268 - acc: 0.8589 - val_loss: 0.4110 - val_acc: 0.7790
Epoch 8/20
15884/15884 [==============================]

In [60]:
model.evaluate(X_val, Y_val)

 992/1000 [============================>.] - ETA: 0s

[0.37677926421165464, 0.86299999999999999]

In [61]:
model.save('model_simple.h5')

Reimplementing resnet

In [99]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense
model = keras.applications.resnet50.ResNet50(include_top=False)
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2, activation='softmax')(x)
model = Model(model.input, x)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
zero_padding2d_7 (ZeroPadding2D) (None, None, None, 3) 0           input_7[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472        zero_padding2d_7[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, None, None, 64 256         conv1[0][0]                      
___________________________________________________________________________________________

In [100]:
for layer in model.layers[:-2]:
    layer.trainable = False

In [101]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [102]:
model.evaluate(X_val, Y_val)

1000/1000 [==============================] - 14s    


[0.7073194031715393, 0.48699999999999999]

In [ ]:
model.fit(X_train, y_train, epochs=15, validation_data=(X_val, Y_val))

Train on 15884 samples, validate on 1000 samples
Epoch 1/15
15884/15884 [==============================] - 184s - loss: 0.2409 - acc: 0.9060 - val_loss: 1.0749 - val_acc: 0.5810
Epoch 2/15
15884/15884 [==============================] - 180s - loss: 0.1893 - acc: 0.9285 - val_loss: 0.1953 - val_acc: 0.9150
Epoch 3/15
15884/15884 [==============================] - 180s - loss: 0.1732 - acc: 0.9335 - val_loss: 0.1711 - val_acc: 0.9260
Epoch 4/15
15884/15884 [==============================] - 180s - loss: 0.1647 - acc: 0.9376 - val_loss: 0.1661 - val_acc: 0.9400
Epoch 5/15
15884/15884 [==============================] - 180s - loss: 0.1578 - acc: 0.9398 - val_loss: 0.2149 - val_acc: 0.9050
Epoch 6/15
15884/15884 [==============================] - 180s - loss: 0.1622 - acc: 0.9387 - val_loss: 0.2559 - val_acc: 0.9030
Epoch 7/15
15884/15884 [==============================] - 180s - loss: 0.1602 - acc: 0.9399 - val_loss: 0.1662 - val_acc: 0.9350
Epoch 8/15
15884/15884 [========================